### Geometrical Features

This notebook shows how to use functions that generate geometrical features of a point cloud.

In [ ]:
import cv2
import numpy as np
import data_loaders as dls
import pointcloud as pc

from viz import plot
from calibration import Calibration, get_fov_pc
from projection import Projection
from normals import estimate_normals_from_spherical_img
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
PATH = '../../'

In [ ]:
gt_img_paths = glob(PATH + 'dataset/KITTI/dataset/data_road/training/gt_image_2/*.png')
gt_img_paths.sort()

velo_paths = glob(PATH + 'dataset/KITTI/dataset/data_road_velodyne/training/velodyne/*.bin')
velo_paths.sort()

calib_paths = glob(PATH + 'dataset/KITTI/dataset/data_road/training/calib/*.txt')
calib_paths.sort()

In [ ]:
id_file = 0
gt_img = cv2.imread(gt_img_paths[id_file])
calib =  Calibration(calib_paths[id_file])
cloud = dls.load_bin_file(velo_paths[id_file])

In [ ]:
# project cloud on ground truth image and get the labels from it
# this function return a point cloud represented as a PyntCloud object
fov_pc = get_fov_pc(cloud[:,:3], calib, gt_img)

In [ ]:
fov_pc.plot(initial_point_size=0.03, use_as_color='ground_truth', cmap=plt.cm.Set1)

In [ ]:
res_az = 100
res_planar = 300
proj = Projection(proj_type='spherical', res_azimuthal=res_az, res_planar=res_planar)

In [ ]:
# we generate a spherical image with radial information
points = fov_pc.xyz
values = np.linalg.norm(points, axis=1)
res_rho = 100
rho_img= proj.project_points_values(points, values, res_values=res_rho, dtype=np.uint16 )

In [ ]:
plt.imshow(rho_img)

In [ ]:
# zoom of the radial image
plt.figure(figsize=(20,4))
plt.imshow(rho_img[151:199, 750:1150])

In [ ]:
# estimate normal from radial image
normals = estimate_normals_from_spherical_img(rho_img, res_rho=res_rho, res_az=res_az, res_planar=res_planar)

In [ ]:
# show normals (X,Y,Z) coordinates are mapped on (R,G,B) channels
plt.figure(figsize=(20,4))
plt.imshow(np.abs(normals[151:199, 750:1150]))

In [ ]:
# we can project back the estimated normals to the points in the point cloud
normals_3D = proj.back_project(points, normals, res_value=1, dtype=np.float)

In [ ]:
fov_pc.points['nx'] = normals_3D[:,0]
fov_pc.points['ny'] = normals_3D[:,1]
fov_pc.points['nz'] = normals_3D[:,2]

In [ ]:
# visualize the values of coordinate z of the normals in 3d
fov_pc.plot(initial_point_size=0.03, use_as_color='nz', cmap=plt.cm.coolwarm)

##### Remark:
Results in normal estimation can be improved interpolating void pixels in rho_img